In [88]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [89]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [90]:
tf.keras.backend.clear_session()

In [91]:
subject = 'Alagoas - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [92]:
data = pd.read_csv('2003_mo_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - Consumo de Cimento (t)
0,2003-1,0.635160,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.294434,8.765721,30.779440,2.590528e+07,1.303625e+06,7.576443,2.359888e+07,30.109
1,2003-2,0.635571,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.296211,8.760196,30.599132,2.593685e+07,1.305020e+06,7.578654,2.360861e+07,27.684
2,2003-3,0.635982,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.297521,8.754671,30.545910,2.596843e+07,1.306414e+06,7.580866,2.361833e+07,26.534
3,2003-4,0.636393,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.298766,8.749145,30.377624,2.600000e+07,1.307809e+06,7.583078,2.362806e+07,25.109
4,2003-5,0.636804,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.299945,8.743620,30.464900,2.603157e+07,1.309203e+06,7.585289,2.363779e+07,25.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587645,NaN,24.556360,NaN,NaN,NaN,NaN,46.546
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587084,NaN,24.501314,NaN,NaN,NaN,NaN,47.781
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.585891,NaN,24.609100,NaN,NaN,NaN,NaN,51.171
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.584366,NaN,24.538488,NaN,NaN,NaN,NaN,49.126


In [93]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-2.106195,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.400856,-0.877266,-0.885888,-1.644358,-1.297696,-1.913990,-1.857572
1,-2.072296,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.374741,-0.879298,-0.900149,-1.627544,-1.242142,-1.888746,-1.832102
2,-2.038397,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.355481,-0.881330,-0.904358,-1.610731,-1.186588,-1.863502,-1.806633
3,-2.004498,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.337169,-0.883362,-0.917668,-1.593917,-1.131034,-1.838257,-1.781163
4,-1.970599,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.319844,-0.885394,-0.910766,-1.577104,-1.075480,-1.813013,-1.755694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.358381,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.990315,1.043483,-1.575976,1.179374,-1.254203,1.153065,1.054229
188,1.351367,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.000562,1.037328,-1.527002,1.172432,-1.239868,1.147901,1.048854
189,1.344353,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.011398,1.031173,-1.472984,1.165489,-1.225534,1.142737,1.043480
190,1.337339,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.028853,1.025017,-1.478716,1.158547,-1.211200,1.137574,1.038106


In [94]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      24.166
1      21.774
2      28.228
3      25.525
4      26.207
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Alagoas - Consumo de Cimento (t), Length: 240, dtype: float64

In [95]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-2.106195,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.400856,-0.877266,-0.885888,-1.644358,-1.297696,-1.913990,-1.857572
1,-2.072296,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.374741,-0.879298,-0.900149,-1.627544,-1.242142,-1.888746,-1.832102
2,-2.038397,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.355481,-0.881330,-0.904358,-1.610731,-1.186588,-1.863502,-1.806633
3,-2.004498,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.337169,-0.883362,-0.917668,-1.593917,-1.131034,-1.838257,-1.781163
4,-1.970599,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.319844,-0.885394,-0.910766,-1.577104,-1.075480,-1.813013,-1.755694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.429306,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.348585,1.320458,-0.588269,1.217771,-1.179058,1.109023,1.141942
158,1.428089,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.330705,1.307307,-0.687207,1.221600,-1.195069,1.116989,1.144540
159,1.426872,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.311609,1.294156,-0.785508,1.225429,-1.211080,1.124956,1.147138
160,1.425655,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.291203,1.281004,-0.883126,1.229259,-1.227092,1.132922,1.149736


In [96]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      24.166
1      21.774
2      28.228
3      25.525
4      26.207
        ...  
157    32.321
158    41.937
159    31.982
160    28.499
161    22.626
Name: Alagoas - Consumo de Cimento (t), Length: 162, dtype: float64

In [97]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [98]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [99]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size : split_index + 1]
test_input

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
126,0.408114,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.871197,1.001453,1.498208,0.788737,-0.193817,0.746412,0.852250
127,0.417817,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.912736,1.019577,1.468126,0.809677,-0.237777,0.767524,0.871358
128,0.427521,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.960653,1.037701,1.431617,0.830616,-0.281737,0.788636,0.890466
129,0.437224,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.006010,1.055825,1.391868,0.851555,-0.325697,0.809748,0.909574
130,0.446928,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.054871,1.073949,1.349013,0.872494,-0.369657,0.830860,0.928682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.358381,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.990315,1.043483,-1.575976,1.179374,-1.254203,1.153065,1.054229
188,1.351367,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.000562,1.037328,-1.527002,1.172432,-1.239868,1.147901,1.048854
189,1.344353,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.011398,1.031173,-1.472984,1.165489,-1.225534,1.142737,1.043480
190,1.337339,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.028853,1.025017,-1.478716,1.158547,-1.211200,1.137574,1.038106


In [100]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [101]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [102]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(72, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(36, activation='tanh'),
        tf.keras.layers.Dense(18, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')    
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
#                         validation_split=0.15,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [103]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [104]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3795642386, 2087550365, 496569726, 2372045401, 416224669, 3663069543, 1933942205, 2203614239, 4096937454, 749877715, 393096443, 3499809632, 4079659732, 668148717, 1751040919, 1220419098, 1061873295, 2719008390, 185904437, 1178572116]


Step: 0 ___________________________________________
val_loss: 43.963279724121094
winner_seed: 3795642386


Step: 1 ___________________________________________
val_loss: 289.8182067871094


Step: 2 ___________________________________________
val_loss: 37.40166473388672
winner_seed: 496569726


Step: 3 ___________________________________________
val_loss: 37.02961730957031
winner_seed: 2372045401


Step: 4 ___________________________________________
val_loss: 289.8181457519531


Step: 5 ___________________________________________
val_loss: 30.303279876708984
winner_seed: 3663069543


Step: 6 ___________________________________________
val_loss: 289.81817626953125


Step: 7 ___________________________________________
val_loss: 37.807037353515625


Step: 8 

In [105]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 116ms/step - loss: 2731.4685 - val_loss: 2726.7471
Epoch 2/10000
4/4 [==============================] - 0s 9ms/step - loss: 2655.6951 - val_loss: 2624.7732
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 2562.0769 - val_loss: 2532.2654
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 2475.1750 - val_loss: 2465.1338
Epoch 5/10000
4/4 [==============================] - 0s 8ms/step - loss: 2414.9534 - val_loss: 2417.9614
Epoch 6/10000
4/4 [==============================] - 0s 8ms/step - loss: 2368.4282 - val_loss: 2382.5364
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 2332.8723 - val_loss: 2355.5137
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 2305.9583 - val_loss: 2334.3562
Epoch 9/10000
4/4 [==============================] - 0s 8ms/step - loss: 2284.5823 - val_loss: 2317.0227
Epoch 10/10000
4/4 [==============================] -

4/4 [==============================] - 0s 8ms/step - loss: 1761.2111 - val_loss: 1815.8895
Epoch 79/10000
4/4 [==============================] - 0s 8ms/step - loss: 1755.3671 - val_loss: 1810.0704
Epoch 80/10000
4/4 [==============================] - 0s 8ms/step - loss: 1749.5974 - val_loss: 1804.2526
Epoch 81/10000
4/4 [==============================] - 0s 8ms/step - loss: 1743.6617 - val_loss: 1798.4972
Epoch 82/10000
4/4 [==============================] - 0s 8ms/step - loss: 1738.0131 - val_loss: 1792.7465
Epoch 83/10000
4/4 [==============================] - 0s 8ms/step - loss: 1732.0033 - val_loss: 1787.0652
Epoch 84/10000
4/4 [==============================] - 0s 8ms/step - loss: 1726.4227 - val_loss: 1781.3182
Epoch 85/10000
4/4 [==============================] - 0s 8ms/step - loss: 1720.5398 - val_loss: 1775.6388
Epoch 86/10000
4/4 [==============================] - 0s 8ms/step - loss: 1714.8845 - val_loss: 1769.9683
Epoch 87/10000
4/4 [==============================] - 0s 8ms/

Epoch 155/10000
4/4 [==============================] - 0s 8ms/step - loss: 1361.2557 - val_loss: 1420.1517
Epoch 156/10000
4/4 [==============================] - 0s 8ms/step - loss: 1356.6415 - val_loss: 1415.6267
Epoch 157/10000
4/4 [==============================] - 0s 8ms/step - loss: 1352.0883 - val_loss: 1411.1477
Epoch 158/10000
4/4 [==============================] - 0s 8ms/step - loss: 1347.5269 - val_loss: 1406.6860
Epoch 159/10000
4/4 [==============================] - 0s 8ms/step - loss: 1343.0823 - val_loss: 1402.1846
Epoch 160/10000
4/4 [==============================] - 0s 8ms/step - loss: 1338.5109 - val_loss: 1397.7155
Epoch 161/10000
4/4 [==============================] - 0s 8ms/step - loss: 1333.9716 - val_loss: 1393.2598
Epoch 162/10000
4/4 [==============================] - 0s 8ms/step - loss: 1329.4846 - val_loss: 1388.7665
Epoch 163/10000
4/4 [==============================] - 0s 8ms/step - loss: 1324.9227 - val_loss: 1384.2732
Epoch 164/10000
4/4 [================

4/4 [==============================] - 0s 8ms/step - loss: 1049.4872 - val_loss: 1112.1298
Epoch 232/10000
4/4 [==============================] - 0s 8ms/step - loss: 1045.9221 - val_loss: 1108.4890
Epoch 233/10000
4/4 [==============================] - 0s 8ms/step - loss: 1042.2045 - val_loss: 1104.9384
Epoch 234/10000
4/4 [==============================] - 0s 8ms/step - loss: 1038.6879 - val_loss: 1101.4326
Epoch 235/10000
4/4 [==============================] - 0s 8ms/step - loss: 1035.0834 - val_loss: 1097.9534
Epoch 236/10000
4/4 [==============================] - 0s 8ms/step - loss: 1031.5880 - val_loss: 1094.4656
Epoch 237/10000
4/4 [==============================] - 0s 8ms/step - loss: 1028.0973 - val_loss: 1090.9771
Epoch 238/10000
4/4 [==============================] - 0s 8ms/step - loss: 1024.5138 - val_loss: 1087.5026
Epoch 239/10000
4/4 [==============================] - 0s 8ms/step - loss: 1021.0259 - val_loss: 1084.0018
Epoch 240/10000
4/4 [==============================] 

4/4 [==============================] - 0s 8ms/step - loss: 803.9849 - val_loss: 869.9503
Epoch 309/10000
4/4 [==============================] - 0s 8ms/step - loss: 801.1741 - val_loss: 867.2017
Epoch 310/10000
4/4 [==============================] - 0s 8ms/step - loss: 798.3754 - val_loss: 864.4661
Epoch 311/10000
4/4 [==============================] - 0s 8ms/step - loss: 795.6354 - val_loss: 861.7249
Epoch 312/10000
4/4 [==============================] - 0s 8ms/step - loss: 792.8391 - val_loss: 859.0056
Epoch 313/10000
4/4 [==============================] - 0s 8ms/step - loss: 790.0515 - val_loss: 856.2861
Epoch 314/10000
4/4 [==============================] - 0s 8ms/step - loss: 787.3054 - val_loss: 853.5803
Epoch 315/10000
4/4 [==============================] - 0s 8ms/step - loss: 784.6018 - val_loss: 850.8566
Epoch 316/10000
4/4 [==============================] - 0s 8ms/step - loss: 781.7443 - val_loss: 848.1939
Epoch 317/10000
4/4 [==============================] - 0s 8ms/step - lo

...

4/4 [==============================] - 0s 8ms/step - loss: 7.2632 - val_loss: 34.3468
Epoch 4069/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.0806 - val_loss: 34.3688
Epoch 4070/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.6309 - val_loss: 36.9032
Epoch 4071/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.7271 - val_loss: 37.0778
Epoch 4072/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.8340 - val_loss: 36.1590
Epoch 4073/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.4777 - val_loss: 36.7525
Epoch 4074/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.3733 - val_loss: 36.7994
Epoch 4075/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.3590 - val_loss: 36.2492
Epoch 4076/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0819 - val_loss: 34.0623
Epoch 4077/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.9443 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 8.5685 - val_loss: 33.0814
Epoch 4148/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.4540 - val_loss: 36.9872
Epoch 4149/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0863 - val_loss: 39.0198
Epoch 4150/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.6556 - val_loss: 39.1739
Epoch 4151/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.6687 - val_loss: 37.5568
Epoch 4152/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0206 - val_loss: 35.5075
Epoch 4153/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.6765 - val_loss: 36.2037
Epoch 4154/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7696 - val_loss: 35.2275
Epoch 4155/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.6132 - val_loss: 33.6879
Epoch 4156/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.9232 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 5.6733 - val_loss: 33.9297
Epoch 4227/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.0822 - val_loss: 33.8043
Epoch 4228/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.8692 - val_loss: 33.6408
Epoch 4229/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.2495 - val_loss: 33.4372
Epoch 4230/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.0659 - val_loss: 33.4369
Epoch 4231/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.4533 - val_loss: 33.5872
Epoch 4232/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5322 - val_loss: 33.8401
Epoch 4233/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.1420 - val_loss: 33.4016
Epoch 4234/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.1255 - val_loss: 33.2875
Epoch 4235/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.8329 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 7.8257 - val_loss: 33.5669
Epoch 4306/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.8167 - val_loss: 34.4948
Epoch 4307/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.3089 - val_loss: 35.0399
Epoch 4308/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.8752 - val_loss: 34.0144
Epoch 4309/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.2957 - val_loss: 33.8511
Epoch 4310/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.1917 - val_loss: 33.6717
Epoch 4311/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.7501 - val_loss: 33.6159
Epoch 4312/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.7823 - val_loss: 34.0451
Epoch 4313/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.6115 - val_loss: 34.2242
Epoch 4314/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.3431 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 5.9816 - val_loss: 36.6501
Epoch 4385/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7651 - val_loss: 37.2809
Epoch 4386/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.3912 - val_loss: 36.9671
Epoch 4387/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.2409 - val_loss: 35.5477
Epoch 4388/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.3066 - val_loss: 34.9554
Epoch 4389/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.0705 - val_loss: 34.3714
Epoch 4390/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.1168 - val_loss: 34.5678
Epoch 4391/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.4989 - val_loss: 35.9082
Epoch 4392/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.1510 - val_loss: 36.2473
Epoch 4393/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.6699 - val_l

In [106]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [107]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,28.468237,28.434587,28.409607,28.388712,28.374111,28.356108,28.330488,28.29958,28.268562,28.241842,28.222383,28.207827,28.204002,28.192144,28.188097,28.185793,28.183134,28.18355,28.1793,28.180435,28.180151,28.172983,28.162283,28.144705,28.12676,28.1035,28.086905,28.090693,28.123108,28.169447
Target,21.34,26.848,28.074,29.941,28.977,31.58,29.991,26.82,29.446,27.666,22.013,28.025,25.648,31.782,27.768,33.073,30.389,27.657,32.388,30.378,30.641,32.337,36.405,22.499,27.961,32.378,32.582,42.703,38.584,36.503
Error,7.128237,1.586588,0.335608,1.552288,0.602888,3.223892,1.660511,1.47958,1.177437,0.575842,6.209383,0.182827,2.556002,3.589855,0.420097,4.887209,2.205866,0.52655,4.2087,2.197565,2.46085,4.164019,8.242716,5.645704,0.16576,4.274498,4.495096,14.612307,10.460892,8.333551


In [108]:
display(mae)
display(mape)

3.6387439

0.11710394

In [109]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [110]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[340.00204]] - Target[330.72099999999995]| =  Error: [[9.281036]]; MAPE:[[0.02806304]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[338.1566]] - Target[360.96500000000003]| =  Error: [[22.80841]]; MAPE:[[0.06318732]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[168.70042]] - Target[210.711]| =  Error: [[42.010574]]; MAPE:[[0.19937533]]


[array([[9.281036]], dtype=float32),
 array([[22.80841]], dtype=float32),
 array([[42.010574]], dtype=float32)]

24.700006

0.09687523